<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/3_Sample_for_Model_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
import re
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
import ast

In [131]:
df_full_ra_rap = pd.read_excel("full_ra_rap.xlsx")

In [132]:
df_full_ra_rap

,מספר תיק,מספר החלטות בתיק,שם הליך,שנת פתיחת ההליך,מספר הליך,סוג הליך,פירוט סוג הליך,תיק חסוי,תאריך אירוע אחרון,משך התיק בימים,...,סוג גורם שיפוטי דלמטה,סוג בית משפט דלמטה,מקום בית משפט דלמטה,הליך מאוחד,שמות ההליכים המאוחדים,צדדים נוספים,מספר תיק מלא,מספר השופטים,ניתנה החלטה כמבוקש הבקשה,Year
0,5821/18,2,"ע""פ 5821/18",2018,"ע""פ 5821/18","ע""פ",ערעור פלילי,0.0,2018-11-22,108,...,['שופט'],['מחוזי '],['חיפה'],NaN,NaN,NaN,NaN,NaN,NaN,2018
1,4217/18,10,"ע""פ 4217/18",2018,"ע""פ 4217/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
2,4344/18,2,"ע""פ 4344/18",2018,"ע""פ 4344/18","ע""פ",ערעור פלילי,0.0,2019-08-18,439,...,['שופט'],['מחוזי '],['מרכז-לוד'],NaN,NaN,NaN,NaN,NaN,NaN,2018
3,1993/18,4,"ע""פ 1993/18",2018,"ע""פ 1993/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,1.0,"[['ע""פ 1993/18', 'ע""פ 2017/18']]",NaN,NaN,NaN,NaN,2018
4,4993/18,4,"ע""פ 4993/18",2018,"ע""פ 4993/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20605,7505/21,1,"רע""פ 7505/21 טל גויחמו נ. מדינת ישראל",2021,"רע""פ 7505/21","רע""פ",רשות ערעור פלילי,0.0,2021-11-11,4,...,"['שופט', 'שופט']","['תעבורה', 'מחוזי ']","['פתח תקווה', 'מרכז-לוד']",NaN,NaN,NaN,NaN,NaN,NaN,2021
20606,513/21,3,"רע""פ 513/21 עידנ יהושע נ. מדינת ישראל",2021,"רע""פ 513/21","רע""פ",רשות ערעור פלילי,0.0,2021-03-22,56,...,"['הרכב', 'שופט']","['מחוזי ', 'שלום']","['באר שבע', 'אשקלון']",NaN,NaN,NaN,NaN,NaN,NaN,2021
20607,225/21,1,"רע""פ 225/21 אלקאר דביר בע""מ נ. מדינת ישראל",2021,"רע""פ 225/21","רע""פ",רשות ערעור פלילי,0.0,2021-02-11,30,...,"['שופט', 'שופט']","['מחוזי ', 'עניינים מקומיים']","['תל אביב', 'תל אביב']",NaN,NaN,NaN,NaN,NaN,NaN,2021
20608,4075/21,1,"רע""פ 4075/21 דב הרמנ נ. מדינת ישראל",2021,"רע""פ 4075/21","רע""פ",רשות ערעור פלילי,0.0,2021-06-10,0,...,"['הרכב', 'שופט']","['מחוזי ', 'שלום']","['תל אביב', 'תל אביב']",NaN,NaN,NaN,NaN,NaN,NaN,2021


# Clean Doc's body - נקיון גוף המסמך

In [133]:
print((df_full_ra_rap["גוף המסמך"].iloc[2]))

['פסק דין בתיק ע"פ 4344/18 בבית המשפט העליון בשבתו כבית משפט לערעורים פליליים ע"פ 4344/18 לפני: כבוד השופט נ\' הנדל כבוד השופט ע\' גרוסקופף כבוד השופט א\' שטיין המערער: אשרף גאבר נגד המשיבה: מדינת ישראל ערעור על פסק דינו של בית המשפט המחוזי מרכז-לוד מיום 17.4.2018 בת"פ 58957-05-17 (השופטת ר\' לורך, ס"נ) תאריך הישיבה: ט"ו בתמוז התשע"ט (18.07.2019) בשם המערער: עו"ד ניר שנידרמן בשם המשיבה: עו"ד עדי שגב פסק דין השופט א\' שטיין: 1. הערעור שלפנינו מופנה אל גזר הדין של בית המשפט המחוזי מרכז-לוד בת"פ 58957-05-17 (השופטת ר\' לורך, ס"נ, כתוארה אז) מיום 17.4.2018, אשר השית על המערער – בעקבות הרשעתו במסגרת הסדר טיעון בגרימת חבלה בכוונה מחמירה, עבירה לפי סעיף 329(א)(1) לחוק העונשין, התשל"ז-1977 (להלן: חוק העונשין), ובנשיאת נשק ותחמושת שלא כדין, עבירה לפי סעיף 144(ב) לחוק העונשין – עונשים כדלקמן: 5 שנות מאסר בפועל החל מיום מעצרו, 28.5.2017; 10 חודשי מאסר על תנאי לבל יעבור עבירת אלימות או עבירת נשק מסוג פשע בתוך 3 שנים מיום שחרורו מהמאסר; 6 חודשי מאסר על תנאי לבל יעבור עבירת אלימות או עבירת נשק מסוג 

In [134]:
# פונקציה להמרת טקסט שמופיע כרשימה למחרוזת רגילה
def convert_list_to_string(text):
    if isinstance(text, str) and text.startswith("[") and text.endswith("]"):
        try:
            # מנסה להמיר את התוכן בתוך הסוגריים לרשימה אמיתית
            text_list = ast.literal_eval(text)
            # איחוד המחרוזות לרצף טקסט אחד
            return ' '.join(text_list)
        except (ValueError, SyntaxError):
            return text
    return text

# פונקציה לניקוי התווים המיותרים
def clean_text(text):
    if isinstance(text, list):
        text = ' '.join(text)  # הפיכת רשימה למחרוזת
    elif isinstance(text, str):
        # הסרת תווי רווח מיותרים ותווים מיוחדים
        text = re.sub(r'\n+', ' ', text)  # הסרת שורות חדשות מרובות
        text = re.sub(r'\\n', '', text)   # הסרת תווי newline \n מהטקסט
        text = re.sub(r'\\xa0', ' ', text)  # הסרת תווי \xa0 מהטקסט
        text = re.sub(r'\s+', ' ', text)  # הסרת רווחים מרובים
        return text.strip()
    return text

# הדפסת שורות לא קריאות לפני המרה
print("שורות לא קריאות לפני המרה:")
print(df_full_ra_rap["גוף המסמך"].head())

# המרה של הטקסטים הלא קריאים לטקסטים קריאים ושמירה על הדאטה המקורי
df_full_ra_rap["גוף המסמך"] = df_full_ra_rap["גוף המסמך"].apply(clean_text)

print("\nשורות לאחר המרה:")
print(df_full_ra_rap["גוף המסמך"].head())

# file_path = 'full_final_df_cleaned.csv'
# df_full_ra_rap.to_csv(file_path, index=False, encoding='utf-8')

# print(f"הקובץ נשמר בהצלחה בנתיב: {file_path}")


שורות לא קריאות לפני המרה:
0    ['פסק דין בתיק ע"פ 5821/18 בבית המשפט העליון ב...
1    ['פסק דין בתיק ע"פ 4217/18 בבית המשפט העליון ב...
2    ['פסק דין בתיק ע"פ 4344/18 בבית המשפט העליון ב...
3    ['פסק דין בתיק ע"פ 1993/18 בבית המשפט העליון ב...
4    ['פסק דין בתיק ע"פ 4993/18 בבית המשפט העליון ב...
Name: גוף המסמך, dtype: object

שורות לאחר המרה:
0    ['פסק דין בתיק ע"פ 5821/18 בבית המשפט העליון ב...
1    ['פסק דין בתיק ע"פ 4217/18 בבית המשפט העליון ב...
2    ['פסק דין בתיק ע"פ 4344/18 בבית המשפט העליון ב...
3    ['פסק דין בתיק ע"פ 1993/18 בבית המשפט העליון ב...
4    ['פסק דין בתיק ע"פ 4993/18 בבית המשפט העליון ב...
Name: גוף המסמך, dtype: object


# DCA Files only

In [135]:
df_full_rap = df_full_ra_rap[df_full_ra_rap["סוג הליך"]=='רע"פ']
df_full_rap

,מספר תיק,מספר החלטות בתיק,שם הליך,שנת פתיחת ההליך,מספר הליך,סוג הליך,פירוט סוג הליך,תיק חסוי,תאריך אירוע אחרון,משך התיק בימים,...,סוג גורם שיפוטי דלמטה,סוג בית משפט דלמטה,מקום בית משפט דלמטה,הליך מאוחד,שמות ההליכים המאוחדים,צדדים נוספים,מספר תיק מלא,מספר השופטים,ניתנה החלטה כמבוקש הבקשה,Year
494,5144/18,1,"רע""פ 5144/18",2018,"רע""פ 5144/18","רע""פ",רשות ערעור פלילי,0.0,2018-07-19,14,...,"['שופט', 'שופט']","['מחוזי ', 'עניינים מקומיים']","['תל אביב', 'בת ים']",NaN,NaN,NaN,NaN,NaN,NaN,2018
495,1746/18,3,"רע""פ 1746/18 אלחננ יהודה פלהיימר נ. מדינת ישראל",2018,"רע""פ 1746/18","רע""פ",רשות ערעור פלילי,0.0,2018-04-29,57,...,"['הרכב', 'שופט']","['מחוזי ', 'שלום']","['מרכז-לוד', 'פתח-תקווה']",NaN,NaN,NaN,NaN,NaN,NaN,2018
496,4262/18,4,"רע""פ 4262/18 רעננ בנ-טובימ נ. מדינת ישראל- עיר...",2018,"רע""פ 4262/18","רע""פ",רשות ערעור פלילי,0.0,2018-08-08,68,...,['שופט'],['מחוזי '],['תל אביב'],NaN,NaN,NaN,NaN,NaN,NaN,2018
497,1654/18,3,"רע""פ 1654/18 לאה מזרחי נ. מדינת ישראל",2018,"רע""פ 1654/18","רע""פ",רשות ערעור פלילי,0.0,2018-07-29,149,...,"['הרכב', 'שופט']","['מחוזי ', 'שלום']","['תל אביב', 'תל אביב']",NaN,NaN,NaN,NaN,NaN,NaN,2018
498,5548/18,4,"רע""פ 5548/18",2018,"רע""פ 5548/18","רע""פ",רשות ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20605,7505/21,1,"רע""פ 7505/21 טל גויחמו נ. מדינת ישראל",2021,"רע""פ 7505/21","רע""פ",רשות ערעור פלילי,0.0,2021-11-11,4,...,"['שופט', 'שופט']","['תעבורה', 'מחוזי ']","['פתח תקווה', 'מרכז-לוד']",NaN,NaN,NaN,NaN,NaN,NaN,2021
20606,513/21,3,"רע""פ 513/21 עידנ יהושע נ. מדינת ישראל",2021,"רע""פ 513/21","רע""פ",רשות ערעור פלילי,0.0,2021-03-22,56,...,"['הרכב', 'שופט']","['מחוזי ', 'שלום']","['באר שבע', 'אשקלון']",NaN,NaN,NaN,NaN,NaN,NaN,2021
20607,225/21,1,"רע""פ 225/21 אלקאר דביר בע""מ נ. מדינת ישראל",2021,"רע""פ 225/21","רע""פ",רשות ערעור פלילי,0.0,2021-02-11,30,...,"['שופט', 'שופט']","['מחוזי ', 'עניינים מקומיים']","['תל אביב', 'תל אביב']",NaN,NaN,NaN,NaN,NaN,NaN,2021
20608,4075/21,1,"רע""פ 4075/21 דב הרמנ נ. מדינת ישראל",2021,"רע""פ 4075/21","רע""פ",רשות ערעור פלילי,0.0,2021-06-10,0,...,"['הרכב', 'שופט']","['מחוזי ', 'שלום']","['תל אביב', 'תל אביב']",NaN,NaN,NaN,NaN,NaN,NaN,2021


In [136]:
years =[2020,2015,2010,2005,2000]

In [137]:
sample_df = df_full_ra_rap[df_full_ra_rap["שנת פתיחת ההליך"].isin(years)]
sample_df

,מספר תיק,מספר החלטות בתיק,שם הליך,שנת פתיחת ההליך,מספר הליך,סוג הליך,פירוט סוג הליך,תיק חסוי,תאריך אירוע אחרון,משך התיק בימים,...,סוג גורם שיפוטי דלמטה,סוג בית משפט דלמטה,מקום בית משפט דלמטה,הליך מאוחד,שמות ההליכים המאוחדים,צדדים נוספים,מספר תיק מלא,מספר השופטים,ניתנה החלטה כמבוקש הבקשה,Year
2430,2888/15,3,"ע""פ 2888/15",2015,"ע""פ 2888/15","ע""פ",ערעור פלילי,0.0,2017-01-02,610,...,['שופט'],['מחוזי '],['מרכז-לוד'],NaN,NaN,NaN,NaN,NaN,NaN,2015
2431,1101/15,5,"ע""פ 1101/15",2015,"ע""פ 1101/15","ע""פ",ערעור פלילי,0.0,2015-10-02,216,...,['שופט'],['מחוזי '],['חיפה'],NaN,NaN,NaN,NaN,NaN,NaN,2015
2432,2805/15,3,"ע""פ 2805/15",2015,"ע""פ 2805/15","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
2433,1961/15,2,"ע""פ 1961/15",2015,"ע""פ 1961/15","ע""פ",ערעור פלילי,0.0,2016-06-07,447,...,['הרכב'],['מחוזי '],['באר שבע'],NaN,NaN,NaN,NaN,NaN,NaN,2015
2434,6963/15,3,"ע""פ 6963/15",2015,"ע""פ 6963/15","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18398,6795/00,3,"רע""פ 6795/00 שינוי התנועה החילונית נ. שר הפנימ",2000,"רע""פ 6795/00","רע""פ",רשות ערעור פלילי,0.0,2001-07-05,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
18399,1492/00,3,"רע""פ 1492/00 ילנה גורליק נ. אלכסנדר גורליק",2000,"רע""פ 1492/00","רע""פ",רשות ערעור פלילי,0.0,2000-07-23,146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
18400,3925/00,1,"רע""פ 3925/00 זרצקי אירינה נ. זרצקי אנדרי",2000,"רע""פ 3925/00","רע""פ",רשות ערעור פלילי,0.0,2000-09-11,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
18417,29/01,4,"ע""פ 29/01 אמינ סעיד חילו נ. מדינת ישראל",2000,"ע""פ 29/01","ע""פ",ערעור פלילי,0.0,2002-12-02,701,...,['הרכב'],['מחוזי '],['חיפה'],NaN,NaN,NaN,NaN,NaN,NaN,2001


In [138]:
sample_df.shape[0]

4346

# Drop null

In [139]:
sample_df[["גוף המסמך"]].dropna(how='any', ignore_index=True,inplace=True)
sample_df[["מספר הליך","שם הליך"]].dropna(how='all', ignore_index=True,inplace=True)
sample_df[["מספר הליך","שם הליך"]].drop_duplicates(inplace=True, ignore_index=True)
sample_df

<ipython-input-139-9f43abb7fdde>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df[["גוף המסמך"]].dropna(how='any', ignore_index=True,inplace=True)
<ipython-input-139-9f43abb7fdde>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df[["מספר הליך","שם הליך"]].dropna(how='all', ignore_index=True,inplace=True)
<ipython-input-139-9f43abb7fdde>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df[["מספר הליך","שם הליך"]

,מספר תיק,מספר החלטות בתיק,שם הליך,שנת פתיחת ההליך,מספר הליך,סוג הליך,פירוט סוג הליך,תיק חסוי,תאריך אירוע אחרון,משך התיק בימים,...,סוג גורם שיפוטי דלמטה,סוג בית משפט דלמטה,מקום בית משפט דלמטה,הליך מאוחד,שמות ההליכים המאוחדים,צדדים נוספים,מספר תיק מלא,מספר השופטים,ניתנה החלטה כמבוקש הבקשה,Year
2430,2888/15,3,"ע""פ 2888/15",2015,"ע""פ 2888/15","ע""פ",ערעור פלילי,0.0,2017-01-02,610,...,['שופט'],['מחוזי '],['מרכז-לוד'],NaN,NaN,NaN,NaN,NaN,NaN,2015
2431,1101/15,5,"ע""פ 1101/15",2015,"ע""פ 1101/15","ע""פ",ערעור פלילי,0.0,2015-10-02,216,...,['שופט'],['מחוזי '],['חיפה'],NaN,NaN,NaN,NaN,NaN,NaN,2015
2432,2805/15,3,"ע""פ 2805/15",2015,"ע""פ 2805/15","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
2433,1961/15,2,"ע""פ 1961/15",2015,"ע""פ 1961/15","ע""פ",ערעור פלילי,0.0,2016-06-07,447,...,['הרכב'],['מחוזי '],['באר שבע'],NaN,NaN,NaN,NaN,NaN,NaN,2015
2434,6963/15,3,"ע""פ 6963/15",2015,"ע""פ 6963/15","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18398,6795/00,3,"רע""פ 6795/00 שינוי התנועה החילונית נ. שר הפנימ",2000,"רע""פ 6795/00","רע""פ",רשות ערעור פלילי,0.0,2001-07-05,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
18399,1492/00,3,"רע""פ 1492/00 ילנה גורליק נ. אלכסנדר גורליק",2000,"רע""פ 1492/00","רע""פ",רשות ערעור פלילי,0.0,2000-07-23,146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
18400,3925/00,1,"רע""פ 3925/00 זרצקי אירינה נ. זרצקי אנדרי",2000,"רע""פ 3925/00","רע""פ",רשות ערעור פלילי,0.0,2000-09-11,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
18417,29/01,4,"ע""פ 29/01 אמינ סעיד חילו נ. מדינת ישראל",2000,"ע""פ 29/01","ע""פ",ערעור פלילי,0.0,2002-12-02,701,...,['הרכב'],['מחוזי '],['חיפה'],NaN,NaN,NaN,NaN,NaN,NaN,2001


In [140]:
cut_sample_df = sample_df[["גוף המסמך","מספר הליך","שם הליך","שנת פתיחת ההליך"]].reset_index()
cut_sample_df

,index,גוף המסמך,מספר הליך,שם הליך,שנת פתיחת ההליך
0,2430,"['החלטה בתיק ע""פ 2888/15 בבית המשפט העליון בשב...","ע""פ 2888/15","ע""פ 2888/15",2015
1,2431,"['פסק דין בתיק ע""פ 1101/15 בבית המשפט העליון ב...","ע""פ 1101/15","ע""פ 1101/15",2015
2,2432,"['פסק דין בתיק ע""פ 2805/15 בבית המשפט העליון ב...","ע""פ 2805/15","ע""פ 2805/15",2015
3,2433,"['פסק דין בתיק ע""פ 1961/15 בבית המשפט העליון ב...","ע""פ 1961/15","ע""פ 1961/15",2015
4,2434,"['פסק דין בתיק ע""פ 6963/15 בבית המשפט העליון ב...","ע""פ 6963/15","ע""פ 6963/15",2015
...,...,...,...,...,...
4341,18398,"[' בבית המשפט העליון בג""ץ6795/00 - ב\' בפני: כ...","רע""פ 6795/00","רע""פ 6795/00 שינוי התנועה החילונית נ. שר הפנימ",2000
4342,18399,"[' בבית המשפט העליון בה""נ 1492/00 בפני כבוד נש...","רע""פ 1492/00","רע""פ 1492/00 ילנה גורליק נ. אלכסנדר גורליק",2000
4343,18400,"[' בבית המשפט העליון בה""נ 3925/00 בפני כבוד נש...","רע""פ 3925/00","רע""פ 3925/00 זרצקי אירינה נ. זרצקי אנדרי",2000
4344,18417,"['פסק דין בתיק ע""פ 29/01 בבית המשפט העליון בשב...","ע""פ 29/01","ע""פ 29/01 אמינ סעיד חילו נ. מדינת ישראל",2000


In [141]:
cut_sample_df.to_csv("cut_sample_df.csv", index=False, encoding='utf-8')

# Cut missing ending files

## FIX THIS!!!


In [142]:
# הגדרת פונקציה לעיצוב טקסט
def format_text(text, line_length=80):
    """
    פורמט טקסט כך שיהיה נוח לקריאה עם שורות שאורכן מוגבל.
    """
    import textwrap
    return "\n".join(textwrap.wrap(text, width=line_length))


In [143]:
# # ביטויים לבדיקה
# keywords = ["העותק כפוף לשינויי עריכה וניסוח", "העתק מתאים", 'ניתנההיום', 'ניתןהיום', 'ניתן היום', 'ניתנה היום',
#             "העתקמתאים", "מזכיר ראשי", "supreme.court.gov.il", "מרכז מידע", "מרכזמידע"]

# # בדיקה אילו שורות מכילות אחד מהביטויים
# rows_with_keyword = cut_sample_df["גוף המסמך"].apply(
#     lambda x: any(keyword in x for keyword in keywords) if isinstance(x, str) else False
# )

# # סינון שורות שלא מכילות את הביטוי
# filtered_data = cut_sample_df[rows_with_keyword]

# # הדפסת מספר השורות שנמחקו
# num_removed = len(cut_sample_df) - len(filtered_data)  # **שונה לחישוב מדויק**
# print(f"\nמספר השורות שנמחקו: {num_removed}")

# # **שינוי לצורך סעיף 6**: בדיקה אם נותרו ערכים חסרים בעמודת "גוף המסמך"
# missing_values_count = filtered_data["גוף המסמך"].isna().sum()
# if missing_values_count > 0:
#     print(f"\nנותרו {missing_values_count} ערכים חסרים בעמודת 'גוף המסמך' לאחר סינון.")
# else:
#     print("\nאין ערכים חסרים בעמודת 'גוף המסמך' לאחר סינון.")

# # שמירת הנתונים המסוננים לקובץ חדש
# filtered_data.to_csv("new_filtered_data.csv", index=False, encoding='utf-8')
# print("\nהנתונים המסוננים נשמרו לקובץ: 'new_filtered_data.csv'")

# # עדכון הטבלה המקורית
# cut_sample_df = filtered_data.copy()  # **שונה להעתקה בטוחה**


In [144]:
# פונקציה לחיתוך הטקסט בהתאם לכללים
def trim_text(text):
    min_length = 500  # אורך מינימלי לשמירה
    last_sen = 30     # מספר תווים להסרה מסוף הטקסט

    # שמירת אורך מקורי לאבחון
    original_length = len(text)

    # הסרת מספר תווים מסוף הטקסט
    text = text[:-last_sen]

    # חיתוך ל-min_length האחרונים אם הטקסט ארוך יותר מהמינימום
    if len(text) > min_length:
        text = text[-min_length:]

    return text

# הסרת שורות עם ערכים חסרים בעמודת "גוף המסמך"
cut_sample_df = cut_sample_df.dropna(subset=["גוף המסמך"]).copy()

# חיתוך הטקסטים בעמודת "גוף המסמך"
cut_sample_df["גוף המסמך חתוך"] = cut_sample_df["גוף המסמך"].apply(trim_text)

# שמירת האינדקס המקורי
cut_sample_df.reset_index(inplace=True, drop=False)  # שונה להבטחת שמירת אינדקס מקורי


In [145]:
cut_sample_df.head()


,level_0,index,גוף המסמך,מספר הליך,שם הליך,שנת פתיחת ההליך,גוף המסמך חתוך
0,0,2430,"['החלטה בתיק ע""פ 2888/15 בבית המשפט העליון בשב...","ע""פ 2888/15","ע""פ 2888/15",2015,בו בפסק הדין – מבטא ביתר שאת את החרטה של המערע...
1,1,2431,"['פסק דין בתיק ע""פ 1101/15 בבית המשפט העליון ב...","ע""פ 1101/15","ע""פ 1101/15",2015,ר בעבירות אלימות ומעמדו של קורבן העבירה – מצאנ...
2,2,2432,"['פסק דין בתיק ע""פ 2805/15 בבית המשפט העליון ב...","ע""פ 2805/15","ע""פ 2805/15",2015,טיפוליות ויש לקוות שיתמיד בכך גם בעתיד – שאז ...
3,3,2433,"['פסק דין בתיק ע""פ 1961/15 בבית המשפט העליון ב...","ע""פ 1961/15","ע""פ 1961/15",2015,עולה כי הוא אינו נוטל אחריות מלאה על מעשיו ונו...
4,4,2434,"['פסק דין בתיק ע""פ 6963/15 בבית המשפט העליון ב...","ע""פ 6963/15","ע""פ 6963/15",2015,"00, על פי החלטת שירות בתי הסוהר, כשברשותו תעוד..."


# Embedding

In [146]:
from transformers import AutoTokenizer, AutoModel
import numpy as np
import joblib
import pandas as pd

# טעינת המודל והטוקנייזר
tokenizer = AutoTokenizer.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")
model = AutoModel.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")

# פונקציה שמקבלת טקסט ומחזירה את האימבדינג כוקטור
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings[0]

# בדיקה אם עמודת "גוף המסמך חתוך" קיימת ומכילה ערכים תקינים
if "גוף המסמך חתוך" not in cut_sample_df.columns or cut_sample_df["גוף המסמך חתוך"].isna().all():
    raise ValueError("עמודת 'גוף המסמך חתוך' חסרה או ריקה. יש לוודא חיתוך טקסטים תקין לפני המשך.")

# חישוב האימבדינגים עבור כל שורה בעמודת "גוף המסמך חתוך"
print("מתחילים לחשב אימבדינגים על גוף המסמך החתוך...")
embeddings_new = np.array([get_embeddings(text) for text in cut_sample_df["גוף המסמך חתוך"]])
print("אימבדינגים חושבו בהצלחה!")


מתחילים לחשב אימבדינגים על גוף המסמך החתוך...
אימבדינגים חושבו בהצלחה!



# Model prediction

In [147]:
# טוען את המודל
classifier = joblib.load("classifier_model.pkl")

print("המודל נטען בהצלחה.")

# חיזוי הסתברויות
print("מתחילים לחזות הסתברויות...")
predicted_probabilities = classifier.predict_proba(embeddings_new)[:, 1]
print("הסתברויות נחזו בהצלחה!")

# הוספת עמודת ההסתברויות
cut_sample_df["predicted_probability"] = predicted_probabilities

# יצירת תחזיות לפי המודל המקורי (ללא שימוש בטרשהולד מותאם)
cut_sample_df["prediction_original_model"] = classifier.predict(embeddings_new)

# **תוספת סעיף 4**: בדיקת התפלגות התחזיות
class_distribution = cut_sample_df["prediction_original_model"].value_counts()
print("\nכמות Class 0 ו-Class 1 (לפי המודל המקורי):")
print(class_distribution)

# בדיקת התפלגות נורמלית
normalized_distribution = cut_sample_df["prediction_original_model"].value_counts(normalize=True)
print("\nהתפלגות נורמלית של Class 0 ו-Class 1 (לפי המודל המקורי):")
print(normalized_distribution)

# שמירה של התחזיות והנתונים לקובץ Excel
output_file = "full_data_with_predictions.xlsx"
cut_sample_df.to_excel(output_file, index=False, engine="openpyxl")
print(f"הטבלה המלאה עם הפרדיקציות נשמרה בקובץ: '{output_file}'")


המודל נטען בהצלחה.
מתחילים לחזות הסתברויות...
הסתברויות נחזו בהצלחה!

כמות Class 0 ו-Class 1 (לפי המודל המקורי):
prediction_original_model
1.0    2288
0.0    2015
Name: count, dtype: int64

התפלגות נורמלית של Class 0 ו-Class 1 (לפי המודל המקורי):
prediction_original_model
1.0    0.531722
0.0    0.468278
Name: proportion, dtype: float64
הטבלה המלאה עם הפרדיקציות נשמרה בקובץ: 'full_data_with_predictions.xlsx'


In [148]:
cut_sample_df

,level_0,index,גוף המסמך,מספר הליך,שם הליך,שנת פתיחת ההליך,גוף המסמך חתוך,predicted_probability,prediction_original_model
0,0,2430,"['החלטה בתיק ע""פ 2888/15 בבית המשפט העליון בשב...","ע""פ 2888/15","ע""פ 2888/15",2015,בו בפסק הדין – מבטא ביתר שאת את החרטה של המערע...,0.997840,1.0
1,1,2431,"['פסק דין בתיק ע""פ 1101/15 בבית המשפט העליון ב...","ע""פ 1101/15","ע""פ 1101/15",2015,ר בעבירות אלימות ומעמדו של קורבן העבירה – מצאנ...,0.999088,1.0
2,2,2432,"['פסק דין בתיק ע""פ 2805/15 בבית המשפט העליון ב...","ע""פ 2805/15","ע""פ 2805/15",2015,טיפוליות ויש לקוות שיתמיד בכך גם בעתיד – שאז ...,0.999993,1.0
3,3,2433,"['פסק דין בתיק ע""פ 1961/15 בבית המשפט העליון ב...","ע""פ 1961/15","ע""פ 1961/15",2015,עולה כי הוא אינו נוטל אחריות מלאה על מעשיו ונו...,0.989550,1.0
4,4,2434,"['פסק דין בתיק ע""פ 6963/15 בבית המשפט העליון ב...","ע""פ 6963/15","ע""פ 6963/15",2015,"00, על פי החלטת שירות בתי הסוהר, כשברשותו תעוד...",0.999891,1.0
...,...,...,...,...,...,...,...,...,...
4298,4341,18398,"[' בבית המשפט העליון בג""ץ6795/00 - ב\' בפני: כ...","רע""פ 6795/00","רע""פ 6795/00 שינוי התנועה החילונית נ. שר הפנימ",2000,"[' בבית המשפט העליון בג""ץ6795/00 - ב\' בפני: כ...",0.424799,0.0
4299,4342,18399,"[' בבית המשפט העליון בה""נ 1492/00 בפני כבוד נש...","רע""פ 1492/00","רע""פ 1492/00 ילנה גורליק נ. אלכסנדר גורליק",2000,"), התשכ""ט1969- המבקשת: ילנהגורליק נגד המשיבים:...",0.021847,0.0
4300,4343,18400,"[' בבית המשפט העליון בה""נ 3925/00 בפני כבוד נש...","רע""פ 3925/00","רע""פ 3925/00 זרצקי אירינה נ. זרצקי אנדרי",2000,"בענייני התרת נישואין(מקריםמיוחדים), התשכ""ט1969...",0.014351,0.0
4301,4344,18417,"['פסק דין בתיק ע""פ 29/01 בבית המשפט העליון בשב...","ע""פ 29/01","ע""פ 29/01 אמינ סעיד חילו נ. מדינת ישראל",2000,מסקנותיו של בית משפט קמא בהקשר זה מקובלות עלינ...,0.999997,1.0



# Analysis

In [149]:
# ווידוא שקיימת עמודת שנה
if "שנת פתיחת ההליך" not in cut_sample_df.columns:
    raise ValueError("עמודת 'שנה' חסרה בדאטה. יש לוודא שהמידע קיים לפני המשך.")

# יצירת טבלת התפלגות
class_distribution_by_year = cut_sample_df.groupby("שנת פתיחת ההליך")["prediction_original_model"].value_counts(normalize=True).unstack(fill_value=0)

# המרה לאחוזים
class_distribution_by_year = class_distribution_by_year * 100

# הדפסת התפלגות הכיתות לפי שנה
print("התפלגות הכיתות לפי תחזיות המודל (%):")
print(class_distribution_by_year)

# שמירה לקובץ Excel
output_file = "class_distribution_by_year.xlsx"
class_distribution_by_year.to_excel(output_file, engine="openpyxl")
# print(f"התפלגות הכיתות נשמרה בקובץ: {output_file}")


התפלגות הכיתות לפי תחזיות המודל (%):
prediction_original_model        0.0        1.0
שנת פתיחת ההליך                                
2000                       71.875000  28.125000
2005                       47.047435  52.952565
2010                       42.596349  57.403651
2015                       40.277778  59.722222
2020                       35.614525  64.385475


In [150]:
data_1 = pd.read_csv("dca_for_classifier.csv")

In [151]:
# בדיקת אורכים מקוריים
original_lengths_train = data_1["גוף המסמך"].str.len()
original_lengths_predict = cut_sample_df["גוף המסמך"].str.len()

print(f"אורך ממוצע של הטקסטים המקוריים באימון: {original_lengths_train.mean():.2f}")
print(f"אורך ממוצע של הטקסטים המקוריים בחיזוי: {original_lengths_predict.mean():.2f}")

# בדיקת אורכים לאחר חיתוך
trimmed_lengths_train = data_1["גוף המסמך"].apply(trim_text).str.len()
trimmed_lengths_predict = cut_sample_df["גוף המסמך"].apply(trim_text).str.len()

print(f"אורך ממוצע של הטקסטים החתוכים באימון: {trimmed_lengths_train.mean():.2f}")
print(f"אורך ממוצע של הטקסטים החתוכים בחיזוי: {trimmed_lengths_predict.mean():.2f}")


אורך ממוצע של הטקסטים המקוריים באימון: 3613.11
אורך ממוצע של הטקסטים המקוריים בחיזוי: 6138.78
אורך ממוצע של הטקסטים החתוכים באימון: 490.08
אורך ממוצע של הטקסטים החתוכים בחיזוי: 494.79


In [152]:
cut_sample_df.columns

Index(['level_0', 'index', 'גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך', 'predicted_probability',
       'prediction_original_model'],
      dtype='object')

In [153]:
# סעיף 3: השוואת נתוני האימון לחיזוי
print("\n--- השוואת נתוני האימון לנתוני החיזוי ---")

# קריאת נתוני האימון (מתוך קובץ האימון)
train_data = pd.read_csv("dca_for_classifier.csv")  # עדכני לנתוני האימון שלך

# השוואת אורכי הטקסטים
train_text_lengths = train_data["גוף המסמך"].str.len()
test_text_lengths = cut_sample_df["גוף המסמך"].str.len()

print(f"אורך ממוצע של טקסטים באימון: {train_text_lengths.mean():.2f}")
print(f"אורך ממוצע של טקסטים בחיזוי: {test_text_lengths.mean():.2f}")

# השוואת אחוז מילות המפתח
keywords = ["העותק כפוף לשינויי עריכה וניסוח", "העתק מתאים", "ניתנה היום", "מרכז מידע"]
train_keyword_percentage = train_data["גוף המסמך"].apply(lambda x: any(kw in x for kw in keywords)).mean()
test_keyword_percentage = cut_sample_df["גוף המסמך"].apply(lambda x: any(kw in x for kw in keywords)).mean()

print(f"אחוז טקסטים עם מילות מפתח באימון: {train_keyword_percentage * 100:.2f}%")
print(f"אחוז טקסטים עם מילות מפתח בחיזוי: {test_keyword_percentage * 100:.2f}%")

# השוואת שיעור Class 1 באימון ובחיזוי
train_class_1_ratio = train_data["binary_outcome"].mean()
predicted_class_1_ratio = cut_sample_df["prediction_original_model"].mean()

print(f"שיעור Class 1 באימון: {train_class_1_ratio * 100:.2f}%")
print(f"שיעור Class 1 בחיזוי: {predicted_class_1_ratio * 100:.2f}%")



--- השוואת נתוני האימון לנתוני החיזוי ---
אורך ממוצע של טקסטים באימון: 3613.11
אורך ממוצע של טקסטים בחיזוי: 6138.78
אחוז טקסטים עם מילות מפתח באימון: 99.59%
אחוז טקסטים עם מילות מפתח בחיזוי: 92.05%
שיעור Class 1 באימון: 6.34%
שיעור Class 1 בחיזוי: 53.17%


In [154]:
# סעיף 4: ווידוא המשקלים של המודל
print("\n--- ווידוא משקלי המודל ---")

# בדיקת משקל הכיתות במודל
if hasattr(classifier, "class_weight"):
    print(f"משקלי הכיתות במודל: {classifier.class_weight}")
else:
    print("לא הוגדרו משקלי כיתות במודל.")

# חישוב שיעור Class 0 ו-Class 1 בחיזוי
predicted_class_distribution = cut_sample_df["prediction_original_model"].value_counts(normalize=True)
print("\nהתפלגות תחזיות המודל:")
print(predicted_class_distribution)

# ווידוא שאכן יש רוב ל-Class 0
if predicted_class_distribution.get(0, 0) <= 0.5:
    print("אזהרה: רוב התחזיות אינן עבור Class 0. ייתכן שהמודל לא מאוזן כהלכה.")



--- ווידוא משקלי המודל ---
משקלי הכיתות במודל: {0: 1, 1: 3}

התפלגות תחזיות המודל:
prediction_original_model
1.0    0.531722
0.0    0.468278
Name: proportion, dtype: float64
אזהרה: רוב התחזיות אינן עבור Class 0. ייתכן שהמודל לא מאוזן כהלכה.


In [155]:
# ווידוא שקיימת עמודת שנה
if "שנת פתיחת ההליך" not in cut_sample_df.columns:
    raise ValueError("עמודת 'שנת פתיחת ההליך' חסרה בדאטה. יש לוודא שהמידע קיים לפני המשך.")

# יצירת קבוצות לפי שנת פתיחת ההליך
grouped_by_year = cut_sample_df.groupby("שנת פתיחת ההליך")

# הדפסת מדגם של 5 מקרים לכל שנה שסווגו כ-Class 1
print("Cases Predicted as Class 1 (Sample of up to 5 cases per year):")
print("=" * 80)

for year, group in grouped_by_year:
    # סינון המקרים שסווגו כ-Class 1 עבור השנה
    class_1_rows = group[group["prediction_original_model"] == 1]

    # בחירת מדגם של עד 5 מקרים
    sampled_class_1_rows = class_1_rows.sample(n=min(5, len(class_1_rows)), random_state=42)

    # הדפסת מקרים עבור השנה
    print(f"\nYear: {year}")
    print("-" * 80)

    for i, (index, row) in enumerate(sampled_class_1_rows.iterrows(), 1):
        print(f"\nCase {i}:")
        print("-" * 80)
        print("Index in Original Data:")
        print(index)
        print("\nPredicted Probability for Class 1:")
        print(row["predicted_probability"])
        print("\nPredicted Label:")
        print(row["prediction_original_model"])
        print("\nDocument Body:")
        formatted_text = "\n".join(row["גוף המסמך"][j:j + 80] for j in range(0, len(row["גוף המסמך"]), 80))
        print(formatted_text)
        print("-" * 80)

    print("=" * 80)


Cases Predicted as Class 1 (Sample of up to 5 cases per year):

Year: 2000
--------------------------------------------------------------------------------

Case 1:
--------------------------------------------------------------------------------
Index in Original Data:
3813

Predicted Probability for Class 1:
0.9684661205936681

Predicted Label:
1.0

Document Body:
[' בבית המשפט העליון בשבתו כבית משפט גבוה לצדק בג"ץ2524/00בפני: כבודהשופט א\' מצ
א כבודהשופט מ\' חשין כבודהשופטת ד\' דורנר העותרים: 1.עיסא מוטיע אל אשהב 2.הוועד 
הציבורי נגד עינויים בישראל נגד המשיב: שירותהבטחון הכללי בשם העותרים: עו"דאנדרה ר
וזנטל בשם המשיב: עו"ד שי ניצן פ ס ק - ד י ן למחוק את העתירה כמבוקש. ניתן היום, כ
ח\' באדר ב\' תש"ס (4.4.2000). ש ו פ ט ש ו פ ט ש ופ ט ת העתק מתאים למקורשמריהוכהן
 - מזכיר ראשיCOURT ']
--------------------------------------------------------------------------------

Case 2:
--------------------------------------------------------------------------------
Index in Original Data:
4002

Pred